In [ ]:
import numpy as np
import pathlib
import json
from tqdm import tqdm
from PIL import Image

In [ ]:
cls_dir = pathlib.Path('/root/dockspace/search_download_images/data/downloads/')
json_dir = pathlib.Path('/root/dockspace/search_download_images/flickr/')
out_dir = pathlib.Path('/root/dockspace/imagenet_candidate_training/data/candididate/')

with open(json_dir / 'chosen_wnids.json') as fn:
    chosen_wnids = json.load(fn)

In [ ]:
chosen_dict = {cur_wnid:i for i, cur_wnid in enumerate(chosen_wnids)}

In [ ]:
npz_list = []

for cur_wnid in chosen_wnids:
    wnid_path = cls_dir / cur_wnid
    npz_list += [npz_file for npz_file in wnid_path.glob('**/*.npz')]
    
    
npz_size_list = []

for npz_file in tqdm(npz_list):
    im_shape = np.load(npz_file)['X_train'].shape
    
    assert im_shape[1:] == (256, 256, 3), "shape {} is not supported".format(im_shape)
    
    npz_size_list.append(im_shape[0])
    
    

In [ ]:
idx_dict = {k:0 for k in chosen_dict.keys()}

In [ ]:
for npz_file in tqdm(npz_list):
    cur_wnid = str(npz_file).split('downloads/')[-1].split('/')[0]
    cur_path = out_dir / cur_wnid
    cur_path.mkdir(parents=True, exist_ok=True)
    
    data = np.load(npz_file)['X_train']
    
    for cur_img in data:
        cur_filename = cur_path / (str(idx_dict[cur_wnid]).zfill(5) + '.jpg')
        pil_image = Image.fromarray(cur_img)
        pil_image.save(cur_filename, format='JPEG', quality=95)
        
        idx_dict[cur_wnid] += 1